# Basic I/O

## Outline

* Intro to impure functions
* Basic IO actions
* Composing IO actions
* The do block
* Recap

In this section, we will introduce the concepts of IO actions. We will begin with introducing these functions and then construct an intuitive but cumbersome way of composing these functions. Then we will introduce a more programmer-friendly way of composing them that hides this awkward construction. At the end, we will recap the import concepts.

## Intro to impure functions
Haskell is a purely functional language. This means that for every function **only** the arguments determine what the outcome will be, these do not change any state. But this is quite a narrow scope for a program. In the real world we want an application to process data from the outside and provide results back to the outside world, it is interactive and should definitely change state outside the scope of the program.

The interactions and changes to state outside this scope are called **side effects**. Say we have a function that fetches the time and logs it into a file. Each time we call this function we give it no inputs and get no outputs back, we only observe the side effect of the file that is changed. So, this function is **impure**. As a recap,

| Pure | Impure |
| --- | --- |
| Always produces the same result when given the same parameter | May produce different results for the same parameter |
| Never has side effect | May have side effects |
| Never alters state | May alter the global state of the program, system, or world |

## Basic IO actions

Haskell strictly separates pure code from the impure world. But we would still like our Haskell programs to alter the world outside our program, it can do this via **IO actions** denoted by `IO a`. All these impure functions include a type that is marked with the `IO` wrapper followed by some type `a`. This marks that it first performs some computation with possible some side effects, then it returns something of type `a`.

As a first example, we have the `putStrLn` function, which has as input a string and returns a type `()`. Together, the resulting return type is of `IO ()`. The `()` type here is the empty tuple, since it has zero elements it can only be one thing, namely nothing.

In [1]:
:t putStrLn --(takes some string and writes it to StdOut)

-- An IO action that perform the action of printing "Hello World".
main :: IO ()
main = putStrLn "Hello World"
main

putStrLn :: String -> IO ()

Hello World

The `IO a` wrapped function are still functions, and so they can be composed with other functions, just keep in mind that the type signatures you are using are correct. Since a combination of an impure function and a pure function still has side effects, the composed function also is an impure function that must use the wrapped marked `IO` as well. We see that we can **never** escape the `IO` wrapper once we used it.

In [2]:
-- A pure function that prepends the string "Hello " to another string
addHello :: String -> String
addHello string = "Hello " ++ string

-- Using the putStrLn function in the definition of an IO action.
newPutStrLn :: String -> IO ()
newPutStrLn name = putStrLn (addHello name)
newPutStrLn "John"

Hello John

Another useful example is the `getLine` action. This will ask for input from the user.

In [13]:
:t getLine --(takes some StdIn and returns a string)

main :: IO String
main = getLine
main

getLine :: IO String

"test"

The last example we will show here is the `listDirectory` which is located in the module `System.Directory`.

In [14]:
import System.Directory (listDirectory) -- Importing some library for the listDirectory function

-- this IO action will list the content of the directory it is given in a list. The "." input refers to the current jupyter working directory (week02)
:t listDirectory
listDirectory "."

listDirectory :: FilePath -> IO [FilePath]

["5-Intro-to-Type-Classes.ipynb","8-Basic-IO.ipynb","7-Creating-Type-Classes.ipynb","1-Introduction-to-haskell.ipynb","2-Functions-Data-Types-and-Signatures.ipynb","3-Pattern-matching-let-where.ipynb","jupyter-tutuorial.ipynb","6-Creating-Types.ipynb","8-Basic-IO(1)(3)(1)(1)(4).ipynb",".ipynb_checkpoints"]

## Composing IO actions
Now that we can use actions to do useful side effects outside our program, we would like to be able to compose two actions, just like with pure functions. This should not be hard, just use another `IO a` as an input as before to get a function of type `IO a -> IO b`. But this construction is not so obvious, this is highlighted in the below example.

In [1]:
-- An IO action that perform the action of printing "Hello". 
someIOType :: IO ()
someIOType = putStrLn "Hello"

-- A naive and wrong function that can combine two IO () actions
combineIO :: IO () -> IO ()    -- A naive and wrong way of composing IO actions
combineIO io = putStrLn "world"
combineIO someIOType

world

Here, the `someIOType` which perform the action of writing “Hello” is not being performed. This is because the input `IO ()` is not being used in the body of the function, so it is not being evaluated. But how can we call this IO action on the right side? To solve this, we introduce the **then** operator given by `>>` and the **bind** operator given by `>>=`. These operators allow for the composition of IO actions. The difference between the two is that the then operator `>>` does not forward the return value of its first action, while the bind operator `>>=` does. So as an overview,

| `>>` | `>>=` |
| --- | --- |
| Composes two IO actions | Composes two IO actions |
| Does not forward the return type | Forwards the return type |

As an example, we first have a look at how the `>>` operator

In [12]:
import System.Directory (listDirectory)

-- A new expression that is of type IO ()
printHello :: IO ()
printHello = putStrLn "Hello " 

-- A new expression that is of type IO ()
printWorld :: IO ()
printWorld = putStrLn "World"

--Combining the above IO actions
printHello >> printWorld

Hello 
World

Here, the `printHello` first performs some side effect and then the `printWorld` action performs its side effect. The arrows of the operator `>>` indicate the flow of the action and in which order they are performed. 

Now for an example of the `>>=` operator

In [7]:
import System.Directory (listDirectory)

-- A new expression that is of type IO ()
getFiles :: IO [FilePath]
getFiles = listDirectory "." 

-- A new expression that is of type IO ()
printFirstFile :: [FilePath] -> IO ()
printFirstFile = print . head 

--Combining the above IO actions with forwarding the 
getFiles >>= printFirstFile

"5-Intro-to-Type-Classes.ipynb"

Here we used `print` which is also an IO action instead of `putStrLn` to show the first element of the list of file paths. This function automatically converts a type to a string and shows it, it saves in syntax.

## The do block

Now that we know how to compose actions, we are going to generalize this with the so called **do block**. To introduce the necessity and reasons of this, we look at a few examples. First, consider the composition of many actions via the then operator `>>`.

In [17]:
--  IO ()              IO String           IO ()
putStrLn "Action one" >> getLine >> putStrLn "Action three"

Action one
Action three

Here each action is **sequentially** performed, the first and last concatenated actions print “Action one” and “Action three” respectively. The middle action asks for input, which performs no output to the console at all. Let's look at how this pattern would work for the bind operator `>>=`.

In [6]:
import System.Directory (listDirectory)

-- A new expression that is of type IO ()
getFiles :: IO [FilePath]
getFiles = listDirectory "." 

-- A new expression that is of type IO ()
printFirstFile :: [FilePath] -> IO ()
printFirstFile = print . head 

-- A new expression that is of type IO ()
printLastFile :: [FilePath] -> IO ()
printLastFile = print . last 

getFiles >>= printFirstFile >>= printLastFile


: 

The compiler gives an error since we are incorrectly matching the types of the functions used. The function `printFirstFile` returns a `()` while the `printLastFile` function expects a `[FilePath]`. Somehow we need to manage how inputs and outputs of these functions are used, ideally we would like the `printLastFile` function to use the output of the `getFiles` function. Achieving this can be done with **anonymous functions** (also called lambda functions). These functions are inline defined function which are imminently applied to an input. They take the form,

In [2]:
(\x -> 2*x + 1) 1 -- Here the lambda function (\x -> 2*x + 1) is applied to the argument 1

3

Using this, we can combine the action while making the inputs **progressively** available to the other actions inline. We do this by adding the actions in order to an anonymous function. The following example achieves what we wanted,

In [2]:
import System.Directory (listDirectory)

-- A new expression that is of type IO ()
getFiles :: IO [FilePath]
getFiles = listDirectory "." 

-- A new expression that is of type IO ()
printFirstFile :: [FilePath] -> IO ()
printFirstFile = print . head 

-- A new expression that is of type IO ()
printLastFile :: [FilePath] -> IO ()
printLastFile = print . last 

getFiles >>= (\x1 -> printFirstFile x1 >>= (\x2 -> printLastFile x1))

"5-Intro-to-Type-Classes.ipynb"
".ipynb_checkpoints"

Here we first forward the output of the `grabFiles` action denoted by `x1` into a lambda function. In this anonymous function, the action `printFirstFile x1` is performed, and its output is forwarded to another lambda function that does not use this output. Instead, it performs the `printLastFile` action with the input `x1`. Note that with this construction, all inputs gradually become available for the last function. Now, clearly this construction is very cumbersome and not practical in real life coding.

That's why to make it easier this whole construction is done by a **do block** which is just some syntactic sugar that does the above construction. This block looks just like an imperative way of programming but importantly, in its core it is still functional. An example of the above composition of actions in a do block becomes,

In [5]:
import System.Directory (listDirectory)

-- A new expression that is of type IO ()
getFiles :: IO [FilePath]
getFiles = listDirectory "." 

-- A new expression that is of type IO ()
printFirstFile :: [FilePath] -> IO ()
printFirstFile = print . head 

-- A new expression that is of type IO ()
printLastFile :: [FilePath] -> IO ()
printLastFile = print . last 

-- The introduction of the do block
do x1 <- getFiles
   printFirstFile x1
   printLastFile x1

"5-Intro-to-Type-Classes.ipynb"
".ipynb_checkpoints"

In a do block, each line performs some action. In the above example, the do block is called by `do` followed by the first action `getFiles`. This action is performed, and its output is stored in the variable `x1`. Then the next action `printFirstFile` is performed with as input the variable `x1`. Lastly, the action `printLastFile` is called with also the input `x1`. This do block  is fundamentally the same as the previous example that used a lambda function. With these do blocks, we can write clearly structured code that performs multiple side effect. Inside a do block, you can also define let bindings define variables in the block. 

In [6]:
do fileList <- getFiles
   let elemOne = fileList !! 1 -- take the
   print elemOne

"8-Basic-IO.ipynb"

## Recap

In this section, we have introduced the concepts of IO actions. 

- Haskell knows two types of functions, pure and impure function, the latter are called IO actions. 
- These types of impure function are wrapped and marked by `IO` and may perform useful side effects. 
- Once a type is wrapped in an IO action, it can never shed this wrapper. 
- IO action can be composed using two operators, the then operator denoted by `>>` and the bind operator denoted by `>>=`.
- The do block let you easily compose multiple action.